# Uniendo los datos

Tomando el contenido de la carpeta `data`, se reunirá la información de las autoridades (de distintas fuentes) según los distintos niveles administrativos que representan. 

In [ ]:
import json
import os
import random

El archivo `alcaldes.json` es probablemente el más completo de todos, ya que posee información de autoridades a distinto nivel para cada uno de los municipios (no comunas) del país. Destacar que la fuente de esos datos posee más información de cada municipio, lo que podría ser útil en versiones futuras de los datos (revisar fuente en README y *commit* respectivo). 

In [ ]:
# Ruta al archivo más completo
ruta_alcaldes = os.path.join("data", "alcaldes.json")
# Cargar los datos del JSON
with open(ruta_alcaldes, "rt", encoding="utf-8") as archivo_alcaldes:
    ALCALDES_JSON = json.load(archivo_alcaldes)

# Correcciones:
# En Chile, hay 345 municipalidades y 346 comunas, dado que la municipalidad 
# de Cabo de Hornos administra a Cabo de Hornos (comuna) y Antártica (comuna)
ALCALDES_JSON["ANTÁRTICA"] = ALCALDES_JSON["CABO DE HORNOS"]
# Las circunscripciones senatoriales no quedaron del todo bien parseadas,
# por lo que hay que eliminar un espacio adicional que quedó en la mitad
for comuna, datos in ALCALDES_JSON.items():
    ALCALDES_JSON[comuna]["circunscripcion"] = " ".join(datos["circunscripcion"].split())

Primero se obtienen los "*tokens*", a los que se les asignará un identificador para poder relacionar los datos entre sí más adelante.

### Comunas (alcaldes y concejales)

In [ ]:
# 345 municipios y 346 comunas (división administrativa menor)
COMUNAS = sorted(ALCALDES_JSON.keys())
ID_COMUNAS = {
    comuna: cid
    for cid, comuna in enumerate(COMUNAS, 1)
}
print(f"{len(COMUNAS)} comunas (alcalde)") # Ver corrección de arriba

### Circunscripciones provinciales (COREs)

In [ ]:
# 64 circunscripciones provinciales (grupos de provincias)
CIRCPROVS = sorted(set(map(lambda c: c.get("area"), ALCALDES_JSON.values())))
ID_CIRCPROVS = {
    circprov: cid
    for cid, circprov in enumerate(CIRCPROVS, 1)
}
print(f"{len(CIRCPROVS)} circunscripciones provinciales (CORE)")

### Provincias (gobernador)

In [ ]:
# 56 provincias (división administrativa intermedia)
PROVINCIAS = sorted(set(map(lambda c: c.get("provincia"), ALCALDES_JSON.values())))
ID_PROVINCIAS = {
    provincia: pid
    for pid, provincia in enumerate(PROVINCIAS, 1)
}
print(f"{len(PROVINCIAS)} provincias (gobernador)")

### Distritos (diputados)

In [ ]:
# 28 distritos electorales (grupos de comunas)
DISTRITOS = sorted(
    set(map(lambda c: c.get("distrito"), ALCALDES_JSON.values())),
    key=lambda d: int(d[9:]),
)
ID_DISTRITOS = {
    distrito: did
    for did, distrito in enumerate(DISTRITOS, 1)
}
print(f"{len(DISTRITOS)} distritos (diputado)")

### Regiones (intendentes)

In [ ]:
# 16 regiones (división administrativa superior)
REGIONES = sorted(set(map(lambda c: c.get("region"), ALCALDES_JSON.values())))
ID_REGIONES = {
    region: rid
    for rid, region in enumerate(REGIONES, 1)
}
print(f"{len(REGIONES)} regiones (intendente)")

### Circunscripciones senatoriales (senadores)

In [ ]:
def roman_to_int(roman):
    roman = roman.replace("IV", "4")
    roman = roman.replace("IX", "9")
    translation = []
    for digit in roman:
        if digit == "I":
            translation.append(1)
        elif digit == "V":
            translation.append(5)
        elif digit == "X":
            translation.append(10)
        elif digit.isdigit():
            translation.append(int(digit))
    return sum(translation)


# 15 ciscrunscripciones senatoriales (grupos de distritos de una misma región)
CIRCSENS = sorted(
    set(map(lambda c: c.get("circunscripcion"), ALCALDES_JSON.values())),
    key=lambda c: roman_to_int(c[16:]),
)
ID_CIRCSENS = {
    circsen: cid
    for cid, circsen in enumerate(CIRCSENS, 1)
}
print(f"{len(CIRCSENS)} circunscripciones (senador)")

## Completando los datos

En esta sección se dejarán cargados todos los archivos a utilizar, de forma de poder ir completando inmediatamente el *output* para cada una de las unidades administrativas o electorales.

### Comunas

```python
{
    # Básico
    "nombre": str,
    "lat": float,
    "lng": float,
    # Referencias
    "codigo": int,
    "otros": {
        "circunscripcion_senatorial": int,
        "circunscripcion_provincial": int,
        "distrito": int,
        "provincia": int,
        "region": int,
    }, 
    # Autoridades
    "autoridades": {
        "alcalde": {
            "nombre": str,
            "partido": str,
            "url_img": str,
        },
        "concejal": [
            {
                "nombre": str,
                "partido": str,
            },
            ...
        ]
    }
}
```

#### Carga datos

In [ ]:
# Ruta al archivo más completo
ruta_coord_comunas = os.path.join("data", "comunas.json")
# Cargar los datos del JSON
with open(ruta_coord_comunas, "rt", encoding="utf-8") as archivo_coord_comunas:
    COORD_COMUNAS_JSON = json.load(archivo_coord_comunas)

# Correcciones:
# Es mejor tener los datos en un formato de diccionario, donde la llave sea
# el nombre de la comuna y el valor los datos restantes. Corrección manual
COORD_COMUNAS = {comuna: None for comuna in COMUNAS}
for datos_comuna in COORD_COMUNAS_JSON:
    nombre_comuna = datos_comuna["nombre"].upper()
    if nombre_comuna in COORD_COMUNAS:
        COORD_COMUNAS[nombre_comuna] = datos_comuna
    elif nombre_comuna == "COIHAIQUE":
        COORD_COMUNAS["COYHAIQUE"] = datos_comuna
    elif nombre_comuna == "O’HIGGINS":
        COORD_COMUNAS["O´HIGGINS"] = datos_comuna
    elif nombre_comuna == "REQUÍNOA":
        COORD_COMUNAS["REQUINOA"] = datos_comuna
    elif nombre_comuna == "SANTIAGO CENTRO":
        COORD_COMUNAS["SANTIAGO"] = datos_comuna
    elif nombre_comuna == "TREGUACO":
        COORD_COMUNAS["TREHUACO"] = datos_comuna
    else:
        print(f"Falta en JSON: {nombre_comuna}")

for comuna, datos in COORD_COMUNAS.items():
    if datos is None:
        print(f"Falta en COORD: {comuna}")


#### Datos completos

In [ ]:
DATOS_COMUNAS = {}
for comuna in COMUNAS:
    datos = ALCALDES_JSON[comuna]
    DATOS_COMUNAS[comuna] = {
        # Básico
        "nombre": comuna,
        "lat": COORD_COMUNAS.get(comuna, {}).get("lat"),
        "lng": COORD_COMUNAS.get(comuna, {}).get("lng"),
        # Referencias
        "codigo": ID_COMUNAS.get(comuna),
        "otros": {
            "circunscripcion_senatorial": ID_CIRCSENS.get(datos.get("circunscripcion")),
            "circunscripcion_provincial": ID_CIRCPROVS.get(datos.get("area")),
            "distrito": ID_DISTRITOS.get(datos.get("distrito")),
            "provincia": ID_PROVINCIAS.get(datos.get("provincia")),
            "region": ID_REGIONES.get(datos.get("region")),
        },
        # Autoridades
        "autoridades": {
            "alcalde": {
                "nombre": datos.get("alcalde", {}).get("nombre"),
                "partido": datos.get("alcalde", {}).get("partido"),
                "url_img": datos.get("alcalde", {}).get("url_img"),
            },
            "concejal": [
                {"nombre": concejal.get("nombre"), "partido": concejal.get("partido")}
                for concejal in datos.get("concejales")
            ],
        }
    }

comuna_sample = random.choice(COMUNAS)
datos_sample = DATOS_COMUNAS[comuna_sample]
print(comuna_sample)
print(json.dumps(datos_sample, indent=4))

In [ ]:
ruta_output_comunas = os.path.join("output", "comunas.json")
with open(ruta_output_comunas, "wt", encoding="utf-8") as output_comunas_json:
    json.dump(
        DATOS_COMUNAS, output_comunas_json,
        sort_keys=False, indent=4,
        separators=(',', ': '), ensure_ascii=False,
)
